In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf

path_file = tf.keras.utils.get_file("shakespeare.txt", 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
# help(tf.keras.utils.get_file)

In [4]:
text = open(path_file, "rb").read().decode(encoding="utf-8")

In [5]:
len(text)

1115394

In [6]:
# This text look like
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
# Bảng chữ cái

vocab = sorted(set(text))

In [8]:
print("Length of vocabulary:", len(vocab))
print(vocab)

Length of vocabulary: 65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


# Bước 1: Xử lý dữ liệu

## Vector hóa text

In [9]:
import numpy as np

# Tạo một dictionary để chứa index và value của vocabulary

char2index = {c:i for i, c in enumerate(vocab)}
# char2index
index2char = np.array(vocab)
index2char

array(['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [10]:
# chuyển về mảng dữ liệu để xử lý

text_as_integer = np.array([char2index[c] for c in text])
text_as_integer

array([18, 47, 56, ..., 45,  8,  0])

In [11]:
print("{\n")
for char, _ in zip(char2index, range(20)):    
    print("{:4s} - {:3d}".format(repr(char), char2index[char]))
print("\n ....}")

{

'\n' -   0
' '  -   1
'!'  -   2
'$'  -   3
'&'  -   4
"'"  -   5
','  -   6
'-'  -   7
'.'  -   8
'3'  -   9
':'  -  10
';'  -  11
'?'  -  12
'A'  -  13
'B'  -  14
'C'  -  15
'D'  -  16
'E'  -  17
'F'  -  18
'G'  -  19

 ....}


In [12]:
# Xem 13 ký tự đầu trong văn bản đã được liên kết với các số

# text[:13]
print("{} ---- characters mapping to integers ----> {}".format(repr(text[:13]), text_as_integer[:13]))

'First Citizen' ---- characters mapping to integers ----> [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [13]:
# Khai báo 
max_length = 100
example_per_epoch = len(text) // (max_length+1)


# Tạo training example / target
# Một kiểu dữ liệu tensor của tf (dùng numpy to get )
character_dataset = tf.data.Dataset.from_tensor_slices(text_as_integer)

for i in character_dataset.take(5):
    print("{} - {}".format(index2char[i.numpy()], i.numpy()))

F - 18
i - 47
r - 56
s - 57
t - 58


In [14]:
# Chuyển các character sang sequences

sequences = character_dataset.batch(max_length + 1, drop_remainder=True)
# help(character_dataset.batch)

for i in sequences.take(2):
    print(repr("".join(index2char[i.numpy()])))

print("\n ------------ \n")

for i in sequences.take(2):
    print("".join(index2char[i.numpy()]))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'

 ------------ 

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [15]:
# For each sequence, duplicate and shift it to form the input and target text 
# by using the map method to apply a simple function to each batch

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    
    return input_text, target_text

dataset = sequences.map(split_input_target)
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [16]:
for input_text, target_text in dataset.take(1):
    print("----> Input text: \n", "".join(index2char[input_text.numpy()]))
    print("----> Target text: \n", "".join(index2char[target_text.numpy()]))

----> Input text: 
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
----> Target text: 
 irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


In [46]:
# Creat training batch

BS = 64

new_dataset = dataset.shuffle(1000).batch(BS, drop_remainder=True)
new_dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [18]:
# Build The Model

# vocab length
vocab_len = len(vocab)

# embedding dimention
embedding_dim = 256

# RNN units
rnn_units = 1024

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, GRU, Dense

def build_model(vocab_len, embedding_dim, rnn_units, batch_size):
    
    model = Sequential([
        Embedding(input_dim=vocab_len, output_dim=embedding_dim, batch_input_shape=[batch_size, None]),
        LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer="glorot_uniform"),
        Dense(vocab_len)
    ])
    
    model.summary()
    return model
# help(LSTM)

In [20]:
model = None
model = build_model(vocab_len, embedding_dim, rnn_units, BS)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Try the model 

for input_example_data, target_example_data in new_dataset.take(1):
    example_batch_predictions = model(input_example_data)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [22]:
sample_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
new_sample_indices = tf.squeeze(sample_indices, axis=-1).numpy()

In [23]:
new_sample_indices

array([51, 20, 54, 44, 35, 57, 28, 12, 14, 42, 21, 61, 63, 50, 21, 44, 59,
       56, 61,  8, 61, 40, 12, 18, 13, 43,  9, 62, 47, 25, 34, 18, 29,  7,
       43, 12,  2, 45,  3, 34, 44, 22, 40, 54, 52, 62, 40, 62, 17, 20,  8,
       25, 46, 14, 17, 34,  4, 15, 12, 18, 58,  5, 38,  8, 57, 36, 46, 13,
       19, 60, 60, 33, 14, 41, 55, 23, 31,  1, 39, 51,  9, 11, 18,  2, 10,
       47, 15, 10, 53, 25, 53,  8, 53, 30, 63, 53, 41, 31, 26, 25])

In [24]:
input_example_data[0]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([57, 58,  1, 47, 58, 57, 43, 50, 44,  0, 13, 40, 53, 60, 43,  1, 39,
        1, 40, 43, 58, 58, 43, 56,  1, 45, 53, 52, 43,  6,  1, 57, 53,  1,
       51, 59, 57, 58,  1, 58, 46, 63,  1, 45, 56, 39, 60, 43,  0, 19, 47,
       60, 43,  1, 61, 39, 63,  1, 58, 53,  1, 61, 46, 39, 58,  5, 57,  1,
       57, 43, 43, 52,  1, 52, 53, 61,  2,  1, 31, 47, 56,  6,  1, 63, 53,
       59,  1, 63, 53, 59, 56, 57, 43, 50, 44,  0, 20, 39, 60, 43])>

In [25]:
print("Input:")
repr("".join(index2char[input_example_data[0]]))

Input:


'"st itself\\nAbove a better gone, so must thy grave\\nGive way to what\'s seen now! Sir, you yourself\\nHave"'

In [26]:
print("Predict:")
repr("".join(index2char[new_sample_indices]))

Predict:


'"mHpfWsP?BdIwylIfurw.wb?FAe3xiMVFQ-e?!g$VfJbpnxbxEH.MhBEV&C?Ft\'Z.sXhAGvvUBcqKS am3;F!:iC:oMo.oRyocSNM"'

In [27]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    
    
help(tf.keras.losses.sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module tensorflow.python.keras.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)
    Computes the sparse categorical crossentropy loss.
    
    Usage:
    
    >>> y_true = [1, 2]
    >>> y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
    >>> loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    >>> assert loss.shape == (2,)
    >>> loss.numpy()
    array([0.0513, 2.303], dtype=float32)
    
    Args:
      y_true: Ground truth values.
      y_pred: The predicted values.
      from_logits: Whether `y_pred` is expected to be a logits tensor. By default,
        we assume that `y_pred` encodes a probability distribution.
      axis: (Optional) Defaults to -1. The dimension along which the entropy is
        computed.
    
    Returns:
      Sparse categorical crossentropy loss value.



In [28]:
# Test loss

example_batch_loss = loss(target_example_data, example_batch_predictions)
# print(example_batch_loss)

print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.175336


In [29]:
# Biên soạn chương trình

model.compile(loss=loss, optimizer="adam")

In [30]:
%cd /content/drive/My Drive/BT_ML

/content/drive/My Drive/BT_ML


In [31]:
# Sử dụng checkpint
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Path save checkpoint
path_checkpint = "./checkpoint"


# Name of checkpoint files
checkpoint_name_files = os.path.join(path_checkpint, "ckpt_{epoch}") 

checkpoint_callback = ModelCheckpoint(filepath=checkpoint_name_files, save_weights_only=True)
# help(ModelCheckpoint)

In [ ]:
# Train the model

model.fit(new_dataset, epochs=20, callbacks=[checkpoint_callback])

In [45]:
new_dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
tf.train.latest_checkpoint(path_checkpint)

'./checkpoint/ckpt_20'

In [32]:
model = build_model(vocab_len, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(path_checkpint))

model.build(tf.TensorShape([1, None]))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Generate text

def generate_text(model, start_string):

  # number character to generate
  num_generate = 1000

  # Converting start string to number (vectoring)
  input_eval = [char2index[c] for c in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to stored the results
  text_generated = []

  
  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size = 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove batch dimension
    predictions = tf.squeeze(predictions, 0)

    # Using categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predictions_idx = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
    
    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    
    input_eval = tf.expand_dims([predictions_idx], 0)

    text_generated.append(index2char[predictions_idx])

  return (start_string + ''.join(text_generated))
  

In [34]:
print(generate_text(model, start_string=u"John: "))

John: that your fair
show themselves to waste his vassal dreads,
Or any other gips of butchers,
Our coot-morrow to my mother, now;
Either we meet, and now in Lady to Call up.'

BENVOLIO:
A' mine eyes advise.

JULIET:
What a matter? speak not thy oath that lie thou must
Thy love hath left me for my brother's drum.

KATHARINA:
Well, let thee both their fortunes to my breast,
With resign too, milled master; though thou know
On him that may be so bad? I'll to the county's pains,
And given two savish woman, and already: speak:
Your fiery good age in the wind;
And e, the man immedience and taste
And sails within their true spirit,
Our steed redress must resolution as I show'd.

SLY:
You are too perish!

Second Murderer:
'He that will find forfessity begin to slay;
Let me my bonys with words by cough and land,
With light attent; we wilt woo enough my cousing steers,
That counts at their full tides for Rovelia
Tears the nobility accept me as she will.
How from Bergarence doth make was some cau